In [3]:
#setting up the DB
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/nobel_prize.db', echo=True) # where is db


from sqlalchemy.ext.declarative import declarative_base #This you have to declare your own schemas like NoSQL and MongoDB
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Enum

class winnerTable(Base): #a child class of Base object
    __tablename__ = 'winner'
    
    #defining the columns
    id = Column(Integer, primary_key=True) 
    category = Column(String)
    name = Column(String)
    nationality = Column(String)
    sex = Column(Enum('male','female'))
    year = Column(Integer)
    
    def __repr__(self):
        return "<winnerTable(name='%s',nationality ='%s', year='%d', sex='%s')" %(self.name, self.category, self.year, self.sex)

Base.metadata.create_all(engine)

2018-10-30 16:29:57,378 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-30 16:29:57,379 INFO sqlalchemy.engine.base.Engine ()
2018-10-30 16:29:57,380 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-30 16:29:57,381 INFO sqlalchemy.engine.base.Engine ()
2018-10-30 16:29:57,381 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winner")
2018-10-30 16:29:57,382 INFO sqlalchemy.engine.base.Engine ()
2018-10-30 16:29:57,383 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winner (
	id INTEGER NOT NULL, 
	category VARCHAR, 
	name VARCHAR, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	year INTEGER, 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2018-10-30 16:29:57,384 INFO sqlalchemy.engine.base.Engine ()
2018-10-30 16:29:57,430 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
#parsing json to a DB

#*makes the session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()


2018-10-30 16:30:22,925 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-30 16:30:22,926 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-30 16:30:22,927 INFO sqlalchemy.engine.base.Engine ('A', 'A name', 'A nation', 'male', 1)
2018-10-30 16:30:22,940 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-30 16:30:22,941 INFO sqlalchemy.engine.base.Engine ('B', 'B name', 'B nation', 'female', 2)
2018-10-30 16:30:22,949 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-30 16:30:22,951 INFO sqlalchemy.engine.base.Engine ('C', 'C name', 'C nation', 'male', 3)
2018-10-30 16:30:22,952 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
import json
with open('../data/nobel_winners.json') as jsonfile:
    reader = json.load(jsonfile)
    for row in reader:
        session.add(winnerTable(**row)) #double stars important, unpacks data
    session.commit()

In [6]:
session.query(winnerTable).count()

result = session.query(winnerTable)
list(result)

2018-10-30 16:30:43,273 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-30 16:30:43,275 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner) AS anon_1
2018-10-30 16:30:43,276 INFO sqlalchemy.engine.base.Engine ()
2018-10-30 16:30:43,279 INFO sqlalchemy.engine.base.Engine SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner
2018-10-30 16:30:43,280 INFO sqlalchemy.engine.base.Engine ()


[<winnerTable(name='A name',nationality ='A', year='1', sex='male'),
 <winnerTable(name='B name',nationality ='B', year='2', sex='female'),
 <winnerTable(name='C name',nationality ='C', year='3', sex='male'),
 <winnerTable(name='A name',nationality ='A', year='1', sex='male'),
 <winnerTable(name='B name',nationality ='B', year='2', sex='female'),
 <winnerTable(name='C name',nationality ='C', year='3', sex='male')]

In [48]:
candidate_A = session.query(winnerTable).get(1) #you cnanot get index 0, starts from 1
# candidate_A.nationality= 'bfdgfdghsdbt434bh'
# candidate_A.name= 'A name changed'

2018-10-30 16:49:07,596 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-30 16:49:07,598 INFO sqlalchemy.engine.base.Engine SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner 
WHERE winner.id = ?
2018-10-30 16:49:07,598 INFO sqlalchemy.engine.base.Engine (1,)


In [12]:
session.query(winnerTable).get(1).name

2018-10-30 16:31:26,621 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-30 16:31:26,622 INFO sqlalchemy.engine.base.Engine SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner 
WHERE winner.id = ?
2018-10-30 16:31:26,623 INFO sqlalchemy.engine.base.Engine (1,)


'A name changed'

In [30]:
session.add(winnerTable(name='D name', nationality= 'D', year ='4',sex ='male')) #already unpacked, no **


In [25]:
list(session.query(winnerTable))

2018-10-30 16:37:00,684 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-30 16:37:00,685 INFO sqlalchemy.engine.base.Engine SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner
2018-10-30 16:37:00,686 INFO sqlalchemy.engine.base.Engine ()


[<winnerTable(name='A name changed',nationality ='A', year='1', sex='male'),
 <winnerTable(name='B name',nationality ='B', year='2', sex='female'),
 <winnerTable(name='C name',nationality ='C', year='3', sex='male'),
 <winnerTable(name='A name',nationality ='A', year='1', sex='male'),
 <winnerTable(name='B name',nationality ='B', year='2', sex='female'),
 <winnerTable(name='C name',nationality ='C', year='3', sex='male'),
 <winnerTable(name='D name',nationality ='None', year='4', sex='male'),
 <winnerTable(name='D name',nationality ='None', year='4', sex='male')]

In [46]:
session.rollback()

2018-10-30 16:48:49,989 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [26]:
session.close()

2018-10-30 16:43:19,655 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [51]:
session.dirty

IdentitySet([])